In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from locker.analysis import *
from locker.data import BaseRate



In [ ]:
rel = Runs() * SecondOrderSignificantPeaks() * StimulusSpikeJitter() * Cells() \
      & dict(stimulus_coeff=1, eod_coeff=0, baseline_coeff=0, refined=1, \
             cell_type='p-unit', am=0, n_harmonics=0) \
      & 'frequency > 0'


### Number of cells

In [ ]:
Cells.proj() & rel

### Number of trials

In [ ]:
Runs().proj() & rel


### Get data

In [ ]:
df = pd.DataFrame(rel.fetch())
df['spread'] = df['stim_std'] / df['eod'] / 2 / np.pi
df['jitter'] = df['stim_std']  # rename to avoid conflict with std function


## Statistical Analysis

### {1,2}-sigma border frequency domain

In [ ]:

print('1 sigma in Frequency domain', np.mean(1 / (2 * np.pi * df.spread)))
print('min sigma in Frequency domain', np.min(1 / (2 * np.pi * df.spread)))
print('max sigma in Frequency domain', np.max(1 / (2 * np.pi * df.spread)))
print('2 sigma in Frequency domain', np.mean(2 / (2 * np.pi * df.spread)))

# 


In [ ]:
print(r"contrast: \rho={0}    p={1}".format(*stats.pearsonr(df.contrast, df.vector_strength)))
df2 = df[df.contrast == 20]
print(r"jitter: \rho={0}    p={1}".format(*stats.pearsonr(df2.jitter, df2.vector_strength)))
print(r"frequency: \rho={0}    p={1}".format(*stats.pearsonr(df2.frequency, df2.vector_strength)))


### Without baseline firing

In [ ]:
glm = smf.glm('vector_strength ~ frequency * jitter + contrast', data=df, 
              family=sm.families.Gamma()).fit()

glm.summary()

In [ ]:
glm.pvalues

### With baseline firing

In [ ]:
glm = smf.glm('vector_strength ~ frequency * jitter + contrast + baseline', data=df, 
              family=sm.families.Gamma()).fit()
glm.summary()

In [ ]:
glm.pvalues